In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import librosa

In [2]:
# LABELDICT:
labeldict={0: 'Sadness',
 1: 'Excited',
 2: 'Happiness',
 3: 'Anger',
 4: 'Frustration',
 5: 'Other'}

In [3]:
# Decode the one hot encoded:
def decode_emotion(one_hot):
    idx_arg_max = np.argmax(one_hot)
    return labeldict[idx_arg_max]

In [4]:
def get_STFT(path):
    data, _ = librosa.load(path, sr=44100)
    STFT = np.abs(librosa.stft(data))
    return STFT

In [5]:
def preprocess_input(path): # Returns a list of x (batch_size, timesteps, feature)
    # Preprocess x:
    x = get_STFT(path)
    # Zero-padding:
    x = keras.preprocessing.sequence.pad_sequences(x, padding="post", maxlen=1497, dtype = np.float16) # maxlen is after discovering the whole training data
    # Reshaping so that the order is not messed up
    x = x.reshape(1, 1025, 1497)
    # Transposing so that we have timesteps in dim 1
    x = x.transpose((0, 2, 1))
    # Convert to tensor and of type tf.float16 for faster operation
    x = tf.convert_to_tensor(x, dtype=tf.float16)
    return x

In [6]:
# Load the model
model = keras.models.load_model('./CNN_LSTM_CONCAT.h5')

2022-05-31 10:15:52.271147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 10:15:52.280372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 10:15:52.280515: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


2022-05-31 10:15:52.281596: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-31 10:15:52.282735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 10:15:52.282849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 10:15:52.282925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [7]:
def predict_emotion(path):
    data = preprocess_input(path)
    predicted = model(data, training=False)
    result = decode_emotion(predicted)
    print(f"The emotion associated with the file is {result}")
    return result

In [13]:
predict_emotion('./DATA/Sadness/Ses01F_impro06_F015.wav')

The emotion associated with the file is Sadness


'Sadness'